In [1]:
import pandas as pd 
pd.set_option("display.max_colwidth", 400)
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk # for text manipulation
from nltk.corpus import stopwords
import preprocessor as p
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [2]:
depression_df = pd.read_csv("./tweets_2019/depression/hate_depression.csv")

In [3]:
#depression_df.info()

In [4]:
depression_df.drop(["content","id","user","outlinks","tcooutlinks","replyCount","retweetCount","likeCount","quoteCount","conversationId","lang","source","sourceUrl","sourceLabel","media","retweetedTweet","quotedTweet","mentionedUsers"] ,axis=1, inplace=True)

In [5]:
depression_df.head(10)

,url,date,renderedContent
0,https://twitter.com/TranscendedBass/status/1212159304532398081,2019-12-31 23:51:13+00:00,"Too many shootings, too many lives lost from evil people with power, too much hate, too much depression, and not nearly enough change to stop that. \n\nI'm gonna be in my apartment hoping every bang I hear tonight is a firework, hoping we as a people can celebrate life without fear"
1,https://twitter.com/bimboluvr3/status/1212153068336418818,2019-12-31 23:26:26+00:00,i hate having to walk back into my depression den after i’ve completely altered myself in the shower
2,https://twitter.com/un_apolo_getic/status/1212150784483319808,2019-12-31 23:17:22+00:00,Seriously. 2019 decided to kick my ass one more time. Now I'm listening to my depression playlist. I hate it here. twitter.com/TarotByBronx/s…
3,https://twitter.com/lordmooh/status/1212149960860454924,2019-12-31 23:14:05+00:00,First 2020 depression... yay... 👌🏻 I hate my life.
4,https://twitter.com/dannyspiders/status/1212137997707173890,2019-12-31 22:26:33+00:00,"I hate that being upfront with your emotional state as an adult is seen as ""attention seeking"" like no I just want you to know I'm not sulking in a corner because I'm mad at you, my brain hasnt had any feel good pharmacy chemicals in a year and its seasonal depression time"
5,https://twitter.com/atdapyramid/status/1212135080929443840,2019-12-31 22:14:58+00:00,my mom says she’s scared that im not eating because of my depression but literally won’t book me anymore therapy sessions i fucking hate my life dawg
6,https://twitter.com/SisiSmurf/status/1212134354543095814,2019-12-31 22:12:05+00:00,I’d have to say 2019 was one of the worst years I’ve ever had. I lost my brother to a heart attack and my niece (who Was like my sister) to depression... Happened 2 months apart within me moving to a different state (which I hate being here). Can I move back home in 2020? https://t.co/U3S377FiGV
7,https://twitter.com/totzofficial/status/1212127724342398976,2019-12-31 21:45:44+00:00,"Every trial, tribulation, and obstacle that came my way this past year tried to corrupt me again... i made it through depression, anxiety, addiction, manipulation, PTSD and just hate in general. Not even the worst of me could get the best of me. Lets do better in 2020. 🥀"
8,https://twitter.com/lalalandlounge/status/1212127437926076420,2019-12-31 21:44:36+00:00,"@realDonaldTrump Don’t like Rush but wish my grandma was alive, she loved the guy and would have liked to see him not be humiliated but uplifted. So much hate for a President who is a narcissist but hey, I have depression. News refuses to report on his accomplishments, so he compliments himself."
9,https://twitter.com/Ayee_Twatch_Dis/status/1212116235221184513,2019-12-31 21:00:05+00:00,"So besides losin my dad I had a good year, Went on a baecae to Miami, Got some clarity on things that have bugged me, let go of a few demons n released some of the hate in my heart, started talkin to my mom more, got closer with family, sorta getting this depression under control"


In [7]:
p.set_options(p.OPT.URL, p.OPT.EMOJI)
p.clean('Preprocessor is #awesome 👍 https://github.com/s/preprocessor')

'Preprocessor is #awesome'

In [8]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
depression_df['pre_proccessed'] = depression_df['renderedContent'].apply(lambda x: p.clean(x)).str.lower()
#depression_df.head(10)

In [9]:
#removing punctuations, numbers and special characters
#regular expression = any character that's not az or AZ or #
depression_df['pre_proccessed'] = depression_df['pre_proccessed'].str.replace("[^a-zA-Z#]", " ")

#removing short words
depression_df['pre_proccessed'] = depression_df['pre_proccessed'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

depression_df.head(10) 

<ipython-input-9-af4d8aec811f>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  depression_df['pre_proccessed'] = depression_df['pre_proccessed'].str.replace("[^a-zA-Z#]", " ")


,url,date,renderedContent,pre_proccessed
0,https://twitter.com/TranscendedBass/status/1212159304532398081,2019-12-31 23:51:13+00:00,"Too many shootings, too many lives lost from evil people with power, too much hate, too much depression, and not nearly enough change to stop that. \n\nI'm gonna be in my apartment hoping every bang I hear tonight is a firework, hoping we as a people can celebrate life without fear",too many shootings too many lives lost from evil people with power too much hate too much depression and not nearly enough change stop that gonna apartment hoping every bang hear tonight firework hoping people can celebrate life without fear
1,https://twitter.com/bimboluvr3/status/1212153068336418818,2019-12-31 23:26:26+00:00,i hate having to walk back into my depression den after i’ve completely altered myself in the shower,hate having walk back into depression den after ive completely altered myself the shower
2,https://twitter.com/un_apolo_getic/status/1212150784483319808,2019-12-31 23:17:22+00:00,Seriously. 2019 decided to kick my ass one more time. Now I'm listening to my depression playlist. I hate it here. twitter.com/TarotByBronx/s…,seriously decided kick ass one more time now listening depression playlist hate here
3,https://twitter.com/lordmooh/status/1212149960860454924,2019-12-31 23:14:05+00:00,First 2020 depression... yay... 👌🏻 I hate my life.,first depression yay hate life
4,https://twitter.com/dannyspiders/status/1212137997707173890,2019-12-31 22:26:33+00:00,"I hate that being upfront with your emotional state as an adult is seen as ""attention seeking"" like no I just want you to know I'm not sulking in a corner because I'm mad at you, my brain hasnt had any feel good pharmacy chemicals in a year and its seasonal depression time",hate that being upfront with your emotional state adult seen attention seeking like just want you know not sulking corner because mad you brain hasnt had any feel good pharmacy chemicals year and its seasonal depression time
5,https://twitter.com/atdapyramid/status/1212135080929443840,2019-12-31 22:14:58+00:00,my mom says she’s scared that im not eating because of my depression but literally won’t book me anymore therapy sessions i fucking hate my life dawg,mom says shes scared that not eating because depression but literally wont book anymore therapy sessions fucking hate life dawg
6,https://twitter.com/SisiSmurf/status/1212134354543095814,2019-12-31 22:12:05+00:00,I’d have to say 2019 was one of the worst years I’ve ever had. I lost my brother to a heart attack and my niece (who Was like my sister) to depression... Happened 2 months apart within me moving to a different state (which I hate being here). Can I move back home in 2020? https://t.co/U3S377FiGV,have say was one the worst years ive ever had lost brother heart attack and niece who was like sister depression happened months apart within moving different state which hate being here can move back home
7,https://twitter.com/totzofficial/status/1212127724342398976,2019-12-31 21:45:44+00:00,"Every trial, tribulation, and obstacle that came my way this past year tried to corrupt me again... i made it through depression, anxiety, addiction, manipulation, PTSD and just hate in general. Not even the worst of me could get the best of me. Lets do better in 2020. 🥀",every trial tribulation and obstacle that came way this past year tried corrupt again made through depression anxiety addiction manipulation ptsd and just hate general not even the worst could get the best lets better
8,https://twitter.com/lalalandlounge/status/1212127437926076420,2019-12-31 21:44:36+00:00,"@realDonaldTrump Don’t like Rush but wish my grandma was alive, she loved the guy and would have liked to see him not be humiliated but uplifted. So much hate for a President who is a narcissist but hey, I have depression. News refuses to report on his accomplishments, so he compliments himself.",dont like rush but wish grandma was alive she loved the guy and woul

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stopwords_english = stopwords.words('english') 

print('Stop words\n')
print(stopwords_english)

Stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

In [12]:
tokenized_tweet = depression_df['pre_proccessed'].apply(lambda x: x.split()) # tokenizing
#tokenized_tweet[:10]

0    [too, many, shootings, too, many, lives, lost, from, evil, people, with, power, too, much, hate, too, much, depression, and, not, nearly, enough, change, stop, that, gonna, apartment, hoping, every, bang, hear, tonight, firework, hoping, people, can, celebrate, life, without, fear]
1                                                                                                                                                                                       [hate, having, walk, back, into, depression, den, after, ive, completely, altered, myself, the, shower]
2                                                                                                                                                                                            [seriously, decided, kick, ass, one, more, time, now, listening, depression, playlist, hate, here]
3                                                                                                                                       

In [15]:
# print(tokenized_tweet[:10])
tweets_clean_dict = {}
for num, sentence in enumerate(tokenized_tweet): # Go through every tweet in the list of tweets 
    tweets_clean = [] # create an empty list to save the cleaned tweet for each tweet 
    for word in sentence: # go through every word inside the "dirty" tweet and keep only the ones you want
        if (word not in stopwords_english): 
            tweets_clean.append(word)
    tweets_clean_dict[num] = tweets_clean  # save the list of "cleaned" words (cleaned tweet) into a dict in the position num (0,1,2,... ,len(tweets))
tweets_clean_dict

{0: ['many',
  'shootings',
  'many',
  'lives',
  'lost',
  'evil',
  'people',
  'power',
  'much',
  'hate',
  'much',
  'depression',
  'nearly',
  'enough',
  'change',
  'stop',
  'gonna',
  'apartment',
  'hoping',
  'every',
  'bang',
  'hear',
  'tonight',
  'firework',
  'hoping',
  'people',
  'celebrate',
  'life',
  'without',
  'fear'],
 1: ['hate',
  'walk',
  'back',
  'depression',
  'den',
  'ive',
  'completely',
  'altered',
  'shower'],
 2: ['seriously',
  'decided',
  'kick',
  'ass',
  'one',
  'time',
  'listening',
  'depression',
  'playlist',
  'hate'],
 3: ['first', 'depression', 'yay', 'hate', 'life'],
 4: ['hate',
  'upfront',
  'emotional',
  'state',
  'adult',
  'seen',
  'attention',
  'seeking',
  'like',
  'want',
  'know',
  'sulking',
  'corner',
  'mad',
  'brain',
  'hasnt',
  'feel',
  'good',
  'pharmacy',
  'chemicals',
  'year',
  'seasonal',
  'depression',
  'time'],
 5: ['mom',
  'says',
  'shes',
  'scared',
  'eating',
  'depression',
  

In [17]:
len(tweets_clean_dict)

1000